# model_GS_LGBM

#### Grid_Search
https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter  
https://www.codexa.net/hyperparameter-tuning-python/

In [1]:
# !jupyter nbconvert --to python model_GS_LGBM.ipynb

[NbConvertApp] Converting notebook model_GS_LGBM.ipynb to python
[NbConvertApp] Writing 2485 bytes to model_GS_LGBM.py


In [2]:
# warningの無視
import warnings
warnings.filterwarnings('ignore')

In [3]:
import pickle

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

import lightgbm as lgb

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

from sklearn.metrics import f1_score

%matplotlib inline

In [4]:
def importances(model):
    """ 変数重要度
    """
    importances = model.feature_importances_
    indices = np.argsort(importances)

    plt.barh(range(len(indices)), importances[indices] , align='center')
    plt.yticks(range(len(indices)), feature_X[indices])
    plt.title('decision tree feature importance')
    plt.xlabel('feature importance')
    plt.ylabel('variable')
    plt.show()

#### =========要変更===========

In [5]:
path = "../data/models/"
modelName= "model_GS_LGBM"

#### ========================

In [6]:
y_Name='LoanStatus'

In [7]:
# pickleで読込む
import pickle

# 前処理済みデータ取得
with open('../data/feature/feature_data.pickle','rb') as f:
    pre_data = pickle.load(f)

In [9]:
pre_data=pre_data.fillna(pre_data.mean())

In [10]:
# 目的変数と説明変数の取得
y=pre_data[y_Name]
X=pre_data.drop(columns=y_Name)

In [11]:
# 訓練データとテストデータの分割
train_valid_X, test_X, train_valid_y, test_y = train_test_split(X, y, test_size=0.2, shuffle=True)

# 訓練データと検証データの分割
train_X, valid_X, train_y, valid_y = train_test_split(train_valid_X, train_valid_y, test_size=0.2, shuffle=True)

In [12]:
"""ハイパーパラメータの max_depth(木の深さ), 
# n_estimators(決定木の数)に関してグリッドサーチを行う
# グリッドサーチに使用するパラメータの値を用意
"""
seed=1
params = {
    'max_depth': [10, 20, 30, 50],
    'n_estimators': [10, 100, 1000]}

#### =========要変更===========

In [13]:
model=lgb.LGBMClassifier(random_state=seed)

#### ========================

In [14]:
grid = GridSearchCV(estimator=model,
    param_grid=params,
    n_jobs=1,
    cv=KFold(5,shuffle=True, random_state=seed),
    scoring='f1',
    verbose=3,
    return_train_score=True)

In [15]:
grid.fit(train_X,  train_y)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[CV 1/5] END ..................max_depth=10, n_estimators=10; total time=   0.1s
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[CV 2/5] END ..................max_depth=10, n_estimators=10; total time=   0.1s
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[CV 3/5] END ..................max_depth=10, n_estimators=10; total time=   0.1s
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[CV 4/5] END ..................max_depth=10, n_estimators=10; total time=   0.1s
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_de

KeyboardInterrupt: 

In [20]:
# 最適解でモデルを作成
model = grid.best_estimator_

In [27]:
# モデルの保存
with open(path + modelName + '.pickle', mode='wb') as f:
    pickle.dump(model, f)

NameError: name 'model' is not defined